Задание 1
Используем файл keywords.csv.
 
Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.
 
Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

Результат классификации запишите в отдельный столбец region.

In [1]:
geo_data = {
 
    'Центр': ['москва', 'тула', 'ярославль'],
 
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
 
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
 
}

In [2]:
import pandas as pd

In [15]:
keywords = pd.read_csv('keywords.csv')

def city_search(keyword):
    for g in geo_data.items():
        for c in g[1]:
            if keyword.find(c) > -1:
                return g[0]
    return 'undefined'
                

keywords['region'] = keywords.keyword.apply(city_search)

keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [16]:
keywords[keywords.region.isin(['Центр', 'Северо-Запад', 'Дальний Восток'])].head()

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток


Задание 2
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг
 
Результат классификации запишите в столбец class

In [20]:
ratings = pd.read_csv('ml-latest/ratings.csv')

def movie_class(rating):
    if rating > 4:
        return 'высокий рейтинг'
    elif rating > 2:
        return 'средний рейтинг'
    else:
        return 'низкий рейтинг'

ratings['class'] = ratings.rating.apply(movie_class)
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,110,1.0,1425941529,низкий рейтинг
1,1,147,4.5,1425942435,высокий рейтинг
2,1,858,5.0,1425941523,высокий рейтинг
3,1,1221,5.0,1425941546,высокий рейтинг
4,1,1246,5.0,1425941556,высокий рейтинг


Задание 3
Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [46]:
ratings_count = ratings.groupby('userId').count()
film_fans = ratings_count[ratings_count['movieId'] >= 100].filter(items=['userId'])

min_ts = ratings.groupby('userId')['timestamp'].min().reset_index()
min_ts.columns = ['userId', 'min_ts']
max_ts = ratings.groupby('userId')['timestamp'].max().reset_index()
max_ts.columns = ['userId', 'max_ts']

joined = film_fans.merge(min_ts, on='userId', how='left').merge(max_ts, on='userId', how='left')
joined['delta'] = joined.max_ts - joined.min_ts
joined.delta.mean()/60/60/24

454.02102537947115

Задание 4
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
 
При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:
 
1. В переменную years запишите список из всех годов с 1950 по 2010.
 
2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
- для каждой строки пройдите по всем годам списка years
- если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
- если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
 
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'
 
4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [63]:
movies = pd.read_csv('ml-latest/movies.csv')

years = []
for y in range(61):
    years.append(y + 1950)

def production_year(title):
    for y in years:
        if title.find(str(y)) > -1:
            return y
    return 1900

movies['year'] = movies.title.apply(production_year)

joined = movies.merge(ratings, on='movieId', how='left')

joined.groupby('year')['rating'].mean().reset_index().sort_values('rating', ascending=False)

,year,rating
8,1957,3.994948
5,1954,3.969399
23,1972,3.965147
13,1962,3.927510
3,1952,3.903648
25,1974,3.891185
26,1975,3.880785
2,1951,3.875303
15,1964,3.872652
28,1977,3.838959
